In [69]:
import pandas as pd
import json
from dotenv import load_dotenv
import os
import openai
import mysql.connector
from sqlalchemy import create_engine
import pymysql
import re

## SYSTEM MASSAGE (PROMPT FOR OPENAI)

In [224]:
OPENAI_PROMPT = """You are an AI assistant that is able to convert natural language into a properly formatted SQL query.

The database you will be querying is called "sakila". Here is the schema of the database:
{schema}

You must always output your answer in JSON format with the following key-value pairs (without the "json" text at the beginning of the output):
{{- "chain of thought": the reasoning or story behind the generated query
- "query": the SQL query that you generated
- "error": an error message if the query is invalid, or null if the query is valid}}

here is the correct output, dont use anything other than this following output example:

{{-"chain_of_thought": "The user is asking for the description of all tables in the sakila database. To achieve this, we would typically use the 'DESCRIBE' or 'SHOW COLUMNS FROM' command in SQL for each table. However, since the user is asking for the description of all tables, we would use the 'SHOW TABLES' command to get the list of tables and then describe each one. But as an AI, I can only generate a single SQL query at a time, so I will provide the SQL command to list all the tables, which is the first step in the process."
-"query": "SELECT title, length FROM film ORDER BY length DESC LIMIT 1;",
-"error": null}}

"""





## GET OPENAI

In [200]:



load_dotenv()

openai.api_type = xxx
openai.api_base = xxx
openai.api_version = xxx
openai.api_key = xxx

def get_completion_from_messages(system_message, user_message, model="gpt-4", temperature=0, max_tokens=500) -> str:

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{user_message}"}
    ]
    
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    
    return response.choices[0].message["content"]


#if __name__ == "__main__":
#    system_message = "You are a helpful assistant"
#    user_message = "Hello, how are you?"
#    print(get_completion_from_messages(system_message, user_message))

## CONNECTING TO DATABASE

In [6]:
cnx = mysql.connector.connect(
  host="xxx",
  user="xxx",
  password="xxx",
  database = 'xxx'
)

In [211]:
cursor=cnx.cursor()
query=("SELECT  TABLE_NAME, COLUMN_NAME, DATA_TYPE, COLUMN_KEY FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA='sakila';")
cursor.execute(query)
data = []
for row in cursor:
  data.append(row)

df = pd.DataFrame(data, columns=cursor.column_names)
df

,TABLE_NAME,COLUMN_NAME,DATA_TYPE,COLUMN_KEY
0,actor,actor_id,smallint,PRI
1,actor,first_name,varchar,
2,actor,last_name,varchar,MUL
3,actor,last_update,timestamp,
4,actor_info,actor_id,smallint,
...,...,...,...,...
127,staff_list,zip code,varchar,
128,store,address_id,smallint,MUL
129,store,last_update,timestamp,
130,store,manager_staff_id,tinyint,UNI


## MAIN APP

In [237]:
# Schema Representation for finances table



# Format the system message with the schema
formatted_system_message = OPENAI_PROMPT.format(schema=df)

# Generate the SQL query from the user message
user_message = "what is the shortest movie?"

# Use GPT-4 to generate the SQL query
response = get_completion_from_messages(formatted_system_message, user_message)

# extract the output with re
pattern = r'\{(?:.|\n)+\}'

# Use re.search() to find the first occurrence of the pattern in the text
match = re.search(pattern, response)

# If a match is found, extract the matched text
if match:
    extracted_text = match.group()

json_response = json.loads(extracted_text)
json_response['query']


'SELECT title, length FROM film ORDER BY length ASC LIMIT 1;'

## EXECUTING QUERY IN DATABASE

In [238]:
cursor=cnx.cursor()
querys=(json_response['query'])
cursor.execute(querys)
data = []
for row in cursor:
  data.append(row)

df = pd.DataFrame(data, columns=cursor.column_names)
df

,title,length
0,ALIEN CENTER,46
